In [5]:
def bool_list_of_length(n):
    """
    Generate a list of `n` True values to represent a boolean list of a specific length.
    :param n: The desired length of the boolean list
    :return: A list of `n` True values (e.g., [True, True, ...])
    """
    if n <= 0:
        return []
    else:
        return [True] + bool_list_of_length(n - 1)

class LCG:
    def __init__(self, seed=[True, False, True]):
        """
        Initialize the LCG with a seed represented as a list of booleans.
        Each boolean corresponds to a bit: True = 1, False = 0.
        """
        # Constants for the LCG algorithm in boolean form
        self.a = [True, False, True, True]  # Example binary multiplier (arbitrary)
        self.c = [True, True, False, True]  # Example binary increment (arbitrary)
        self.m = bool_list_of_length(31)    # Modulus as a boolean list of 31 bits
        
        # Set the state using the seed (already a boolean list)
        self.state = seed

    def next(self):
        """
        Update the state using the LCG formula:
        state = (a * state + c) % m
        """
        # Perform the LCG calculation using boolean list operations
        state_times_a = bool_list_multiply(self.state, self.a)
        print(f"state_times_a (after multiplication): {state_times_a}")  # Debugging: Print state after multiplication
        
        state_times_a_plus_c = bool_list_add(state_times_a, self.c)
        print(f"state_times_a_plus_c (after addition): {state_times_a_plus_c}")  # Debugging: Print state after addition
        
        self.state = bool_list_modulus(state_times_a_plus_c, self.m)
        print(f"Updated state (after modulus): {self.state}")  # Debugging: Print state after modulus
        
        return self.state

    def random(self):
        """
        Generate a random float between 0 and 1 by treating the boolean list as a binary fraction.
        :return: Random float in [0, 1)
        """
        random_value = sum(1 << i for i, bit in enumerate(reversed(self.state)) if bit) / (2 ** len(self.m))
        print(f"Random value generated: {random_value}")  # Debugging: print the random value
        return random_value

# Function to perform binary addition on boolean lists
def bool_list_add(a, b):
    max_len = max(len(a), len(b))
    a = [False] * (max_len - len(a)) + a  # Pad with False for equal length
    b = [False] * (max_len - len(b)) + b

    result = []
    carry = False
    for i in range(max_len - 1, -1, -1):
        sum_bit = a[i] != b[i] != carry
        carry = (a[i] and b[i]) or (carry and (a[i] or b[i]))
        result.insert(0, sum_bit)
    if carry:
        result.insert(0, True)
    return result[-max_len:]  # Return only the significant bits

# Function to perform binary multiplication on boolean lists
def bool_list_multiply(a, b):
    result = [False] * (len(a) + len(b))  # Result will be at most len(a) + len(b) bits
    for i in range(len(b) - 1, -1, -1):
        if b[i]:
            shifted_a = a + [False] * (len(b) - 1 - i)
            result = bool_list_add(result, shifted_a)
    return result

# Function to simulate modulus with a boolean list
def bool_list_modulus(a, m):
    return a[-len(m):] if len(a) > len(m) else a

# Function to mimic random.choice using LCG
def random_choice(collection, lcg):
    index = int(lcg.random() * len(collection))
    print(f"Selected index: {index}")  # Debugging: print the selected index
    return collection[index]

# Example usage
if __name__ == "__main__":
    # Define the True and False collections to represent the probabilities
    true_collection = [True, True, True, True, True, True, True]  # Represents 70% probability (7 True values)
    false_collection = [False, False, False]  # Represents 30% probability (3 False values)

    # Create the boolean collection
    bool_collection = true_collection + false_collection

    # Initialize LCG with a boolean seed
    bool_seed = [True, False, True]  # Boolean seed for the LCG
    lcg = LCG(seed=bool_seed)

    # Track how many times True and False are selected
    true_count = 0
    false_count = 0

    # Call selected_value 100 times
    for _ in range(100):
        selected_value = random_choice(bool_collection, lcg)
        if selected_value:
            true_count += 1
        else:
            false_count += 1

    # Display the results
    print(f"True was selected {true_count} times.")
    print(f"False was selected {false_count} times.")


Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2.3283064365386963e-09
Selected index: 0
Random value generated: 2

In [6]:
from functools import partial
from abc import ABC, abstractmethod

# Define a Category class (abstract base)
class Category(ABC):
    @abstractmethod
    def compose(self, other):
        pass

    @abstractmethod
    def identity(self):
        pass

# QuantumState as an object in the category
class QuantumState(Category):
    def __init__(self, label):
        # label as an identifier (|0⟩, |1⟩, superpositions)
        self.label = label

    def __repr__(self):
        return f"QuantumState({self.label})"

    # Identity morphism for a quantum state
    def identity(self):
        return self

    # Composition of two states (superposition operation)
    def compose(self, other):
        return Superposition(self, other)

# Define a morphism (quantum gate) in the category
class QuantumGate(Category):
    def __init__(self, function):
        # Function encodes how the gate transforms states
        self.function = function

    def __repr__(self):
        return f"QuantumGate({self.function.__name__})"

    # Identity gate (doing nothing to the state)
    def identity(self):
        return QuantumGate(lambda x: x)

    # Compose two gates (morphisms)
    def compose(self, other):
        return QuantumGate(lambda x: self.function(other.function(x)))

    # Apply gate to a quantum state (functor action)
    def apply(self, state):
        return self.function(state)

# Superposition is a specific composition of states
class Superposition(QuantumState):
    def __init__(self, state1, state2):
        self.state1 = state1
        self.state2 = state2

    def __repr__(self):
        return f"Superposition({self.state1}, {self.state2})"

    # The identity for superposition is itself
    def identity(self):
        return self

    # Compose two superpositions (could generalize this)
    def compose(self, other):
        return Superposition(self, other)

# A Hadamard gate implementation (as a morphism)
def hadamard_gate(state):
    if state.label == "|0⟩":
        return Superposition(QuantumState("|0⟩"), QuantumState("|1⟩"))
    elif state.label == "|1⟩":
        return Superposition(QuantumState("|0⟩"), QuantumState("|1⟩"))
    else:
        return state

# Pauli-X gate (quantum NOT gate)
def pauli_x_gate(state):
    if state.label == "|0⟩":
        return QuantumState("|1⟩")
    elif state.label == "|1⟩":
        return QuantumState("|0⟩")
    else:
        return state

# Examples of quantum gates (Hadamard and Pauli-X)
hadamard = QuantumGate(hadamard_gate)
pauli_x = QuantumGate(pauli_x_gate)

# Example usage
if __name__ == "__main__":
    # Define two basic quantum states |0⟩ and |1⟩
    state_0 = QuantumState("|0⟩")
    state_1 = QuantumState("|1⟩")

    # Superpose two states |0⟩ and |1⟩
    superposed = state_0.compose(state_1)
    print(f"Superposition: {superposed}")

    # Apply Hadamard gate to |0⟩
    new_state = hadamard.apply(state_0)
    print(f"Hadamard applied to |0⟩: {new_state}")

    # Apply Pauli-X gate to |0⟩
    new_state_x = pauli_x.apply(state_0)
    print(f"Pauli-X applied to |0⟩: {new_state_x}")


Superposition: Superposition(QuantumState(|0⟩), QuantumState(|1⟩))
Hadamard applied to |0⟩: Superposition(QuantumState(|0⟩), QuantumState(|1⟩))
Pauli-X applied to |0⟩: QuantumState(|1⟩)
